In [1]:
import tensorflow as tf
import numpy as np
tf.__version__

'1.10.0'

In [2]:
x_data = np.matrix([
#    Users  |     Movies     |    Movie Ratings   | Time | Last Movies Rated
#   A  B  C | TI  NH  SW  ST | TI   NH   SW   ST  |      | TI  NH  SW  ST
    [1, 0, 0,  1,  0,  0,  0,   0.3, 0.3, 0.3, 0,     13,   0,  0,  0,  0 ],
    [1, 0, 0,  0,  1,  0,  0,   0.3, 0.3, 0.3, 0,     14,   1,  0,  0,  0 ],
    [1, 0, 0,  0,  0,  1,  0,   0.3, 0.3, 0.3, 0,     16,   0,  1,  0,  0 ],
    [0, 1, 0,  0,  0,  1,  0,   0,   0,   0.5, 0.5,   5,    0,  0,  0,  0 ],
    [0, 1, 0,  0,  0,  0,  1,   0,   0,   0.5, 0.5,   8,    0,  0,  1,  0 ],
    [0, 0, 1,  1,  0,  0,  0,   0.5, 0,   0.5, 0,     9,    0,  0,  0,  0 ],
    [0, 0, 1,  0,  0,  1,  0,   0.5, 0,   0.5, 0,     12,   1,  0,  0,  0 ]
])
# ratings
y_data = np.array([5, 3, 1, 4, 5, 1, 5])
# Let's add an axis to make tensoflow happy.
y_data.shape += (1, )

print(x_data.shape)
print(y_data.shape)

(7, 16)
(7, 1)


In [6]:
n, p = x_data.shape
# number of latent factors
k = 5
# design matrix
X = tf.placeholder('float', shape=[n, p])
# target vector
y = tf.placeholder('float', shape=[n, 1])
# bias and weights
w0 = tf.Variable(tf.zeros([1]))
W = tf.Variable(tf.zeros([p]))
# interaction factors, randomly initialized 
V = tf.Variable(tf.random_normal([k, p], stddev=0.01))
# estimate of y, initialized to 0.
y_hat = tf.Variable(tf.zeros([n, 1]))

linear_terms = tf.add(w0, tf.reduce_sum(tf.multiply(W, X), 1, keep_dims=True))

interactions = (tf.multiply(0.5, tf.reduce_sum(tf.subtract(tf.pow( tf.matmul(X, tf.transpose(V)), 2),
                                                      tf.matmul(tf.pow(X, 2), tf.transpose(tf.pow(V, 2)))),1, keep_dims=True)))


y_hat = tf.add(linear_terms, interactions)

lambda_w = tf.constant(0.001, name="lambda_w")
lambda_v = tf.constant(0.001, name="lambda_v")

l2_norm = tf.reduce_sum(tf.add(tf.multiply(lambda_w, tf.pow(W, 2)), tf.multiply(lambda_v, tf.pow(V, 2))))

error = tf.reduce_mean(tf.square(tf.subtract(y, y_hat)))
loss = tf.add(error, l2_norm)

eta = tf.constant(0.1)
optimizer = tf.train.AdagradOptimizer(eta).minimize(loss)

epochs = 1000

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for e in range(epochs):
        indices = np.arange(n)
        np.random.shuffle(indices)
        x_data, y_data = x_data[indices], y_data[indices]
        sess.run(optimizer, feed_dict={X: x_data, y: y_data})

        print('MSE: ', sess.run(error, feed_dict={X: x_data, y: y_data}))
        print('Loss (regularized error):', sess.run(l2_norm, feed_dict={X: x_data, y: y_data}))
        print('Predictions:', sess.run(y_hat, feed_dict={X: x_data, y: y_data}))
        print('Learnt weights:', sess.run(W, feed_dict={X: x_data, y: y_data}))
        print('Learnt factors:', sess.run(V, feed_dict={X: x_data, y: y_data}))

MSE:  6.200141
Loss (regularized error): 0.00078783155
Predictions: [[2.2502737 ]
 [1.4431779 ]
 [1.9995646 ]
 [2.1432278 ]
 [1.405962  ]
 [0.97794604]
 [1.8136964 ]]
Learnt weights: [0.09925239 0.09925264 0.09834231 0.09833789 0.0938239  0.09939418
 0.09763661 0.09816746 0.09252849 0.09941673 0.0971072  0.09999904
 0.09905782 0.06714965 0.09763661 0.        ]
Learnt factors: [[-0.0349171  -0.02407259 -0.03465685 -0.02840431 -0.01282445 -0.03264687
  -0.0196564  -0.03018572 -0.01471556 -0.04302935 -0.01254991 -0.09452567
  -0.03668318 -0.01236212 -0.02169501  0.01665276]
 [ 0.03755647  0.0243118   0.02591703  0.02360563  0.01461401  0.02969944
   0.03004999  0.0217854   0.01159155  0.04880134  0.01846263  0.09660977
   0.04271194  0.00057572  0.02351119 -0.00412058]
 [ 0.06877448  0.02859494  0.02880696  0.06725962  0.02209451  0.02975727
   0.02210783  0.04846366  0.03478051  0.0536657   0.02091765 -0.08759408
   0.03941663  0.02398752  0.01666521  0.00533745]
 [ 0.00829347 -0.0026790

Loss (regularized error): 0.0032587221
Predictions: [[4.3776464]
 [3.9967651]
 [5.132376 ]
 [2.4233198]
 [3.7650676]
 [3.3538194]
 [1.2980661]]
Learnt weights: [ 0.11885365  0.2617262   0.0240456   0.10245693 -0.01531189  0.2312582
  0.1401937   0.06900445  0.11012618  0.15185603  0.25670254  0.12283924
  0.1719873  -0.2242881   0.1401937   0.        ]
Learnt factors: [[-0.07059672 -0.2856532  -0.00244308 -0.13253532  0.08139758 -0.1327598
  -0.05191188 -0.03512138 -0.02419209 -0.08977829 -0.27186945 -0.18059707
  -0.12051641  0.2881529  -0.05386501  0.01650591]
 [ 0.07406273  0.2839407  -0.00588103  0.13135788 -0.08003341  0.13147773
   0.06249084  0.02746771  0.02245557  0.096486    0.27653188  0.18583742
   0.12678471 -0.30187333  0.05621202 -0.00408425]
 [ 0.03662149 -0.03783778 -0.0051186   0.05457954  0.05190943 -0.0603433
  -0.01200529  0.02787425  0.03346037  0.00799846 -0.04280127 -0.03430738
  -0.00599182  0.15434207 -0.01704613  0.00529038]
 [-0.01720024 -0.221455    0.03113

MSE:  0.037337195
Loss (regularized error): 0.00386071
Predictions: [[3.0895941]
 [4.9760313]
 [4.8092976]
 [4.76987  ]
 [1.0450672]
 [1.3769444]
 [4.1389723]]
Learnt weights: [ 0.14832215  0.24423382 -0.07196408  0.03111622 -0.08961838  0.256568
  0.13450888  0.03170755  0.13738027  0.12007403  0.23804621  0.11597578
  0.20447062 -0.27039912  0.13450888  0.        ]
Learnt factors: [[-0.10657465 -0.24090165  0.08748962 -0.12682597  0.17923254 -0.18416621
  -0.03988254 -0.02071679 -0.05979948 -0.06547479 -0.22717398 -0.22490028
  -0.16925909  0.3432414  -0.04181808  0.01629837]
 [ 0.11071827  0.2391582  -0.09721247  0.1270891  -0.17977354  0.18396266
   0.05043335  0.01295983  0.05873483  0.07207508  0.23167619  0.23241754
   0.17592867 -0.35804334  0.04420996 -0.00403289]
 [-0.00881371 -0.0111364   0.02391052  0.06873272  0.22108853 -0.13584833
  -0.01027417  0.02222251 -0.00521803  0.01453725 -0.01738071 -0.09781707
  -0.07230875  0.28169826 -0.01533832  0.00522387]
 [-0.05729605 -0.

Predictions: [[4.05647  ]
 [1.0966136]
 [1.1180073]
 [3.1083913]
 [5.0725384]
 [5.053602 ]
 [5.052522 ]]
Learnt weights: [ 0.15491499  0.23313305 -0.09002594  0.01424376 -0.1028571   0.25528863
  0.13760738  0.02494261  0.14212851  0.11082283  0.22570607  0.11493312
  0.21225739 -0.27360758  0.13760738  0.        ]
Learnt factors: [[-0.11689465 -0.21667826  0.1041979  -0.12378868  0.20074964 -0.19330002
  -0.05534761 -0.02046383 -0.06998401 -0.05950205 -0.20287253 -0.23605786
  -0.1842152   0.35195968 -0.05729388  0.0160731 ]
 [ 0.12127325  0.21500455 -0.11425465  0.12426887 -0.2017471   0.19334455
   0.06599075  0.01272713  0.0691527   0.06607059  0.20736353  0.24412313
   0.19110282 -0.3669514   0.05973289 -0.00397715]
 [-0.03241235  0.00849166  0.03393675  0.07280058  0.28167337 -0.16198218
  -0.02082629  0.01502201 -0.02593907  0.01742937  0.00083327 -0.12168535
  -0.10168745  0.31465483 -0.02583073  0.00515166]
 [-0.06998336 -0.1440245   0.14142297 -0.06138878  0.2201619  -0.17375

Learnt factors: [[-0.1089045  -0.21994482  0.10597773 -0.1204728   0.21383901 -0.19126083
  -0.05472306 -0.01465451 -0.06178752 -0.05489541 -0.20591478 -0.21982828
  -0.18564536  0.36948973 -0.05665961  0.01596163]
 [ 0.11330174  0.21848506 -0.11592443  0.12113596 -0.21492179  0.19141422
   0.06553768  0.00697654  0.0609792   0.06155189  0.21058317  0.22758067
   0.19264846 -0.38466522  0.05930929 -0.00394957]
 [-0.03162713  0.00154272  0.02467118  0.06823171  0.2938266  -0.16877131
  -0.02567051  0.01319968 -0.02275885  0.01405824 -0.00486015 -0.06075186
  -0.11169423  0.33465046 -0.0306383   0.00511593]
 [-0.06337443 -0.1486595   0.14122559 -0.06051482  0.23310855 -0.17339532
  -0.03536808  0.01660237 -0.0225618  -0.01343437 -0.14400351 -0.18016003
  -0.15358554  0.3780463  -0.03629363  0.00870472]
 [ 0.13240077  0.20226002 -0.15387864  0.15228818 -0.22470897  0.16669995
   0.04407028 -0.00669889  0.05908195  0.03816916  0.20744175  0.20979273
   0.17110212 -0.37255168  0.04957091 -0

...

MSE:  4.9055126e-07
Loss (regularized error): 0.0033191647
Predictions: [[4.999824 ]
 [3.0002344]
 [3.9989095]
 [1.0004791]
 [4.9995527]
 [4.999177 ]
 [1.0010256]]
Learnt weights: [ 0.13517925  0.19446024 -0.08261633 -0.00109981 -0.07471549  0.225274
  0.09444635  0.01327797  0.08195903  0.09117654  0.1510205   0.11673708
  0.18077932 -0.18387975  0.09444635  0.        ]
Learnt factors: [[-0.11617834 -0.22074342  0.11729176 -0.11981823  0.22099213 -0.20047703
  -0.05888499 -0.01373179 -0.06743724 -0.05643542 -0.19632025 -0.23758976
  -0.19342312  0.37216276 -0.06063974  0.00939354]
 [ 0.12067379  0.21988052 -0.12718952  0.1208009  -0.22241402  0.20087497
   0.06891709  0.00628373  0.06685391  0.06300939  0.20111522  0.24601205
   0.20043965 -0.3873051   0.06325924 -0.00232435]
 [-0.03860495 -0.00989036  0.02422706  0.05814483  0.27324426 -0.1724141
  -0.02631344  0.00796522 -0.02657828  0.0065643  -0.01395573 -0.09028787
  -0.11908896  0.31607515 -0.03034267  0.00301077]
 [-0.07164007 

MSE:  4.8881077e-07
Loss (regularized error): 0.003299815
Predictions: [[4.999551 ]
 [4.999177 ]
 [1.0010229]
 [3.000231 ]
 [4.999822 ]
 [3.9989104]
 [1.0004733]]
Learnt weights: [ 0.13451672  0.19347085 -0.0820999  -0.00114121 -0.07384569  0.22426909
  0.09341499  0.01310718  0.0806435   0.09073834  0.1493814   0.11672875
  0.17969379 -0.18169497  0.09341499  0.        ]
Learnt factors: [[-0.11613674 -0.22087386  0.11730105 -0.1197704   0.22089572 -0.20046566
  -0.05888282 -0.01367821 -0.06735374 -0.05647465 -0.19618055 -0.23775779
  -0.19334789  0.37209466 -0.06063276  0.00925784]
 [ 0.12063126  0.22002614 -0.12719339  0.12075723 -0.22232214  0.20086655
   0.0688938   0.00623562  0.06677292  0.06304535  0.2009781   0.24618603
   0.20036222 -0.38723373  0.06325106 -0.00229077]
 [-0.03855097 -0.01028899  0.0242219   0.05789972  0.27236384 -0.17219846
  -0.02627507  0.00796786 -0.02645405  0.00644174 -0.01427052 -0.09052307
  -0.11893332  0.31541467 -0.03028149  0.00296728]
 [-0.0716084

MSE:  4.8609917e-07
Loss (regularized error): 0.0032791586
Predictions: [[4.9991837]
 [1.0004802]
 [4.9995565]
 [1.0010241]
 [4.9998264]
 [3.0002375]
 [3.9989166]]
Learnt weights: [ 0.13380043  0.19240166 -0.08154274 -0.00118565 -0.07291281  0.22318241
  0.09230716  0.0129234   0.07923791  0.09026457  0.14762191  0.11671976
  0.17852147 -0.17935131  0.09230716  0.        ]
Learnt factors: [[-0.11609174 -0.22101378  0.11731074 -0.11971844  0.22079092 -0.20045342
  -0.05887983 -0.01362029 -0.0672633  -0.05651717 -0.1960278  -0.23793988
  -0.19326618  0.37201998 -0.06062457  0.00911257]
 [ 0.12058528  0.2201826  -0.1271972   0.12070979 -0.22222236  0.20085756
   0.06886794  0.00618365  0.06668519  0.06308433  0.20082805  0.2463746
   0.20027821 -0.3871555   0.06324156 -0.00225483]
 [-0.03849261 -0.01072009  0.02421649  0.05763403  0.27141106 -0.17196502
  -0.02623347  0.00797069 -0.02631993  0.00630887 -0.01460987 -0.09077767
  -0.11876459  0.3146986  -0.03021525  0.00292071]
 [-0.0715744

Predictions: [[4.9995575]
 [4.999184 ]
 [3.998919 ]
 [1.0010217]
 [3.0002365]
 [4.9998264]
 [1.0004793]]
Learnt weights: [ 0.13320166  0.19150849 -0.08107813 -0.0012227  -0.07213929  0.222274
  0.09138706  0.01277039  0.07807629  0.08986856  0.14616156  0.11671212
  0.17754278 -0.1774075   0.09138706  0.        ]
Learnt factors: [[-0.11605387 -0.22112973  0.11731865 -0.11967449  0.2207032  -0.20044315
  -0.05887671 -0.01357172 -0.06718737 -0.0565526  -0.1958986  -0.23809226
  -0.19319749  0.3719569  -0.06061709  0.0089923 ]
 [ 0.12054657  0.22031242 -0.1272002   0.12066964 -0.22213876  0.20084997
   0.06884565  0.00614007  0.0666115   0.06311679  0.20070115  0.24653246
   0.20020755 -0.38708937  0.06323297 -0.00222507]
 [-0.03844352 -0.01108036  0.02421217  0.05741173  0.27061433 -0.17176956
  -0.02619854  0.00797325 -0.02620769  0.00619774 -0.01489259 -0.09099078
  -0.11862306  0.3140989  -0.03015974  0.00288216]
 [-0.07154568 -0.1532551   0.1493651  -0.06368075  0.23771915 -0.1825891

MSE:  4.827328e-07
Loss (regularized error): 0.0032421406
Predictions: [[1.0010204]
 [4.999184 ]
 [3.998921 ]
 [3.0002356]
 [1.000473 ]
 [4.999555 ]
 [4.999825 ]]
Learnt weights: [ 0.13249245  0.19045112 -0.08052908 -0.00126633 -0.07123023  0.22119781
  0.09030402  0.01258989  0.07671589  0.08939943  0.14444375  0.11670297
  0.17638485 -0.17512274  0.09030402  0.        ]
Learnt factors: [[-0.11600883 -0.22126591  0.11732778 -0.11962216  0.22059892 -0.20043084
  -0.05887222 -0.01351405 -0.0670971  -0.05659451 -0.19574378 -0.23827316
  -0.1931157   0.37188143 -0.06060743  0.0088512 ]
 [ 0.12050052  0.22046508 -0.12720348  0.12062182 -0.22203942  0.20084085
   0.0688184   0.00608834  0.06652392  0.06315514  0.20054904  0.24671973
   0.20012346 -0.3870103   0.06322199 -0.00219015]
 [-0.03838515 -0.0115071   0.02420735  0.05714784  0.26966995 -0.17153753
  -0.02615694  0.00797644 -0.02607471  0.00606593 -0.01522643 -0.09124353
  -0.11845487  0.31338713 -0.03009377  0.00283694]
 [-0.0715115

In [ ]:
vacab_size = 
embedding_size = 
v_embeedings = tf.get_variable("v_embeddings", [, 5], initializer=tf.zeros_initializer()) 

In [ ]:
a = 